In [2]:
%pip install pinecone pinecone-text pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pinecone
print(pinecone.__version__)

7.3.0


In [ ]:
api_key = "pinecone_db_api_key"

In [3]:
import os
from pinecone import Pinecone, ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"

pc = Pinecone(api_key = api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension= 384,
        spec = ServerlessSpec(cloud = "aws", region = "us-east-1"),
        metric= "dotproduct"  #sparse values supported only with dotproduct
    )

In [4]:
index = pc.Index(index_name)
print(index)

f:\Desktop\Langchain\langenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
print(embeddings)


model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} query_encode_kwargs={} multi_process=False show_progress=False


In [6]:
from pinecone_text.sparse import BM25Encoder
bm25_encoder = BM25Encoder().default()
print(bm25_encoder)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alok\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [ ]:
sentences = [
    "The cat sat on the mat.",
    "Dogs are great pets.",
    "I love programming in Python.",
    "The quick brown fox jumps over the lazy dog.",
    "Artificial Intelligence is the future."
]
bm25_encoder.fit(sentences)
# store in json file
bm25_encoder.dump("bm25_values.json")
# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 5/5 [00:00<00:00, 42.11it/s]


In [8]:
from langchain_community.retrievers import  PineconeHybridSearchRetriever
retriever = PineconeHybridSearchRetriever(index = index, embeddings = embeddings, sparse_encoder=bm25_encoder)

In [9]:
print(retriever)

embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False) sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000001FE1153EE90> index=<pinecone.db_data.index.Index object at 0x000001FE11047D50>


In [10]:
retriever.add_texts(sentences, ids = [f"doc{i}" for i in range(len(sentences))])

100%|██████████| 1/1 [00:05<00:00,  5.47s/it]


In [11]:
retriever.invoke("What pets are good to have?")

[Document(metadata={'score': 0.4434582}, page_content='Dogs are great pets.'),
 Document(metadata={'score': 0.161552429}, page_content='The quick brown fox jumps over the lazy dog.'),
 Document(metadata={'score': 0.0947384834}, page_content='The cat sat on the mat.'),
 Document(metadata={'score': 0.0555243492}, page_content='I love programming in Python.')]